In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from collections import Counter, namedtuple
from itertools import chain
import ftfy
import pickle
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from string import Template

import numpy as np
import pandas as pd
import spacy

In [ ]:
from src.data_utils import create_news_stats_dataset, create_inverted_news_dict, get_teams
from src.spacy_utils import load_spacy_model
from src.generate_templates import GenerateTemplates, get_context, get_context_tags, text_normalization

In [ ]:
nlp = load_spacy_model('../../data/teams_aliases.txt', '../../data/player_news.csv')

In [ ]:
#news_stats_df = create_news_stats_dataset('../../data/player_news.csv', '../../data/football_db_player_stats.csv',
#                                          '../../data/news_and_stats.csv')
news_stats_df = pd.read_csv('../../data/news_and_stats.csv')
news_stats_df = news_stats_df[lambda df: df['week'] < 13]

In [ ]:
TemplateModel = namedtuple('TemplateModel', ['vectorizer', 'classifier'])
def noop(d):
    return d

with open('../../models/ngram_nb.pkl', 'rb') as f:
    TemplateModel = pickle.load(f)

In [ ]:
template_generator = GenerateTemplates(nlp, '../../data/teams_aliases.txt', vectorizer=TemplateModel.vectorizer, clf=TemplateModel.classifier)

In [ ]:
_ = template_generator.template_transformer(news_stats_df, '../../data/temp_templates.csv')

# WIP

In [ ]:
news_dict = news_stats_df.loc[13].to_dict()
inverted_news_dict = create_inverted_news_dict(news_dict, 
                                               template_generator.data_cols, 
                                               template_generator.team_id_dict,
                                               template_generator.id_team_dict)

normalized_text = text_normalization(news_dict['report'])

doc = nlp(normalized_text)

doc = template_generator.template_tagging(doc, inverted_news_dict, training=False)
news_template = template_generator.doc_to_template(doc)

In [ ]:
print(doc)
print()
print(news_template.template)

In [ ]:
# R2
def r2(chunks, i):
    tag_types = [None, 'player', 'game', 'passing', 'rushing', 'receptions']
    encoding = [0] * len(tag_types)

    for idx, chunk in enumerate(chunks):
        if idx == i:
            break
        encoding[tag_types.index(chunk._.record_type)] = 1

    encoding
    
# R3
def r3(chunks, i):
    i_tag_type = chunks[i].record_type
    if i_tag_type is None:
        return 0
    else:
        for idx, chunk in enumerate(chunks):
            if chunk._.record_type == i_tag_type:
                return 1
        return 0
    
# R4
def r4(chunk_i):
    chunk_values
    for token in chunk_i:
        if token._.template_tag is not None:
            pass

In [ ]:
output_templates = pd.read_csv('../../data/nbmodel_templates.csv')
output_templates.rename(index=str, columns={'reports': 'report'}, inplace=True)
asdf = pd.merge(news_stats_df, output_templates, how='right', on='report')

In [ ]:
def get_data_col_types(text):
    template_regex = re.compile(r'\$\{([_a-z][_a-z0-9]*)\}')
    tags = re.findall(template_regex, text)
    return [template_generator.data_col_to_type[tag] for tag in tags]